<a href="https://colab.research.google.com/github/ping-Mel/ERV-V2-Assignments/blob/main/session11/s11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ERA-V2-Architecture==0.1.3

In [2]:
from ERA_V2_Architecture.model import ResNet
from ERA_V2_Architecture.model import BasicBlock
from ERA_V2_Architecture.model import Models

!pip install torchsummary
from __future__ import print_function
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np


# Now you can use the classes and functions defined in models.py


In [5]:
net = ResNet(BasicBlock, [2, 2, 2, 2])

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

y = net(torch.randn(1, 3, 32, 32))
print(y.size())

torch.Size([1, 10])
